In [ ]:
import pandas as pd
import copy

# Node의 부모 Class
class Node:
    def __init__(self, node_name):
        self.node_name = node_name

# Edge의 부모 Class
class Edge:
    def __init__(self, origin, dest):
        self.origin = origin
        self.dest = dest

# Airport Class
class Airport(Node):
    def __init__(self, airport_name):
        super().__init__(airport_name)

# Flight Class
class Flight(Edge):
    def __init__(self, index, origin, dest, origin_date, origin_time, dest_date, dest_time, flight_duration, flight_type):
        super().__init__(origin, dest)
        self.index = index
        self.origin_date = origin_date
        self.origin_time = origin_time
        self.dest_date = dest_date
        self.dest_time = dest_time
        self.flight_duration = flight_duration
        self.flight_type = flight_type

    def print(self):
        print(self.index, self.origin, self.origin_date, self.origin_time, self.dest,
              self.dest_date, self.dest_time, self.flight_duration, self.flight_type)

# Stack Node의 구조
class Stackitem:
    def __init__(self, flight: Flight, flights_sequence: list = None):
        if flights_sequence:
            self.flight = flight
            flights_sequence.append(flight.index)
            self.flights_sequence = flights_sequence
            # self.time = time + dt.timedelta(flight.flight_duration)
        else:
            self.flight = flight
            self.flights_sequence = [flight.index]
            # self.time = flight.flight_duration;

In [ ]:
# Initial Setting - Test
flight_graph = dict()
flight_excel = pd.read_excel("CGinputEX.xlsx")
flight_excel.columns = ['INDEX', 'ORIGIN', 'ORIGIN_DATE','ORIGIN_TIME', 'DEST','DEST_DATE', 'DEST_TIME', 'FLIGHT_DURATION','AIRCRAFT_TYPE' ]
airports = ["HNL","ORD","LAX"]

# Graph insert Node
for airport_name in airports :
    flight_graph[airport_name] = list()


# Graph insert Edge
for i in range(flight_excel.shape[0]) :
    flight_graph[flight_excel.loc[i]['ORIGIN']].append(Flight(flight_excel.loc[i]['INDEX'], flight_excel.loc[i]['ORIGIN'], flight_excel.loc[i]['DEST'],
                                                              flight_excel.loc[i]['ORIGIN_DATE'], flight_excel.loc[i]['ORIGIN_TIME'], flight_excel.loc[i]['DEST_DATE'], flight_excel.loc[i]['DEST_TIME'],
                                                              flight_excel.loc[i]['FLIGHT_DURATION'], flight_excel.loc[i]['AIRCRAFT_TYPE']))

# print(flight_excel)
# for airport_name in airports : 
#     print(airport_name, "- start - ---------Edge----------")
#     for flight in flight_graph[airport_name] :
#         flight.print()

In [ ]:
pairing_set = list()

# 여기서 Node 시작에 따라 끝나는 경우는
# 만나는 Base가 같은경우가 있고,
# 만약 끝이 Base가 아닌데 갈 곳이 없는 경우가 있다.
# 그렇다면 모든 pairing의 모든 경우를 넣어도 모든 경우가 나온다!

for airport in airports : 
    # Base 이름 넣을려고 만듦
    pairing_set.append(["-" * 10, airport, "-" * 10])
    
    for flight in flight_graph[airport]:
        # 항공편, 항공편의 순서를 넣는다
        stack_temp = [Stackitem(flight)]
        while stack_temp:
            # 현재 진행 중인 항공편의 정보
            flights_information_now = stack_temp.pop()
            
            # 만약 해당 항공편이 진행됨으로서 base로 간다면 해당 항공편은 더 진행할 필요가 없음
            pairing_set.append(copy.deepcopy(flights_information_now.flights_sequence))
            if (flights_information_now.flight.dest == flight.origin):
                continue
            else :
                pairing_set[-1].append("DeadHead")

            # Search Space Extend
            for flights_next in flight_graph[flights_information_now.flight.dest]:
                # 항공기 기종이 같은지? 시간이 가능한지? (날짜가 같다면 시간까지 비교, 다르다면 날짜 비교)
                flight_type_equal = (flights_information_now.flight.flight_type == flights_next.flight_type)
                flight_time_possible = (flights_information_now.flight.dest_date == flights_next.origin_date and flights_information_now.flight.dest_time < flights_next.origin_time) or (flights_information_now.flight.dest_date < flights_next.origin_date)
                
                # 기종이 다르거나 시간이 불가능하면 해당 항공편은 Search에 사용하지 않음
                if (not flight_type_equal or not flight_time_possible) :
                    continue
                
                # 기종이 같고, 시간이 가능하다면 Search를 위해 stack에 추가(항공편의 순서까지)
                temp_list = copy.deepcopy(flights_information_now.flights_sequence)
                if (flights_information_now.flight.dest_date < flights_next.origin_date) :
                    for i in range(0, int(flights_next.origin_date.day - flights_information_now.flight.dest_date.day)) :
                        temp_list.append("Layover")
                        
                stack_temp.append(Stackitem(flights_next, temp_list))

# 첫번째 방안 : 모든 항공편들의 순서를 저장한다
# 그렇다면 만들 수 있는(DeadHead, Layover, Feasible) Pairing의 모든 경우가 나온다.
# 하지만 이럴때는 시간이 n^n까지 나올 수 있는 것으로 보인다.
# 여기서 경우의 수를 줄이는 조건을 넣을 때
# 첫번째 - 항공기 종류
# 두번째 - 전 항공편의 도착시간보다 현 항공편의 출발시간이 느려야한다
# 세번째 - Crew의 Trun Time이 MMT보다 크거나 같아야 한다.(Slack Time은 어떻게 될려나?)
# feasible pairing은 좀 많다. -> 그럼 DeadHead가 존재하는 pairing은 넣어야하는 것인가?
# 지연전파? 존재한다?

# stack에 넣을 때 모든 정보를 넣어버린다면? 오히려 쉬워진다. 하지만 메모리를 많이 쓰므로 최적화가 필요함(조건을 생각해야 함)

for temp in pairing_set:
    print(temp)
